# Model Training & Building

In this step, we are ready to do some experiments and select the best model for production use.

![Training](./images/04-ModelDeployment.jpg)



## Training

**Input** : Registered Model in Model Registry

**Output** : Container Instance / Endpoint (Web Service)

### 1. Load Registered Models

In [12]:
from azureml.core import Workspace
from azureml.core.model import Model
import sklearn

# Get workspace from run context
workspace = Workspace.from_config()

# Load Data
nb_model = Model(workspace, 'ProductReview-NaiveBayes')
vectorizor = Model(workspace, 'ProductReview-CountVector')


### 2. Prepare score.py

In [71]:
%%writefile inference/score.py
import json, os, joblib
from azureml.core.model import Model

def init(): 
  global vec, clf
  print(Model.get_model_path('ProductReview-NaiveBayes'))
  vec = joblib.load(Model.get_model_path('ProductReview-CountVector'))
  clf = joblib.load(Model.get_model_path('ProductReview-NaiveBayes'))

def run(data): 
  input_data = json.loads(data)['data'] 
  fitted_data = vec.transform(input_data)
  pred = clf.predict(fitted_data)
  return json.dumps(pred.tolist())


Overwriting inference/score.py


### 3. Inference Config

In [72]:
from azureml.core.model import InferenceConfig
from azureml.core import Environment

service_name = 'product-review-service'
env = Environment.get(workspace=workspace, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

inference_config = InferenceConfig(entry_script='score.py', 
                            source_directory='./inference',
                            environment=env)

Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'


### 4. Deployment Config

In [73]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

service = Model.deploy(
    workspace,
    name = service_name,
    models=[nb_model, vectorizor],
    inference_config= inference_config,
    deployment_config= deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

### 5. Test Endpoint - Call REST API

In [100]:
import requests, json

uri = service.scoring_uri
headers = {"Content-Type": "application/json"}
comments = """
"I've been using Dreamweaver (and it's predecessor Macromedia's UltraDev) for many years.  
For someone who is an experienced web designer, this course is a high-level review of the CS5 version of Dreamweaver,
 but it doesn't go into a great enough level of detail to find it very useful.\n\nOn the other hand, 
 this is a great tool for someone who is a relative novice at web design.  
 It starts off with a basic overview of HTML and continues through the concepts necessary to build a modern web site.  
 Someone who goes through this course should exit with enough knowledge to create something that does what 
 you want it do do...within reason.  Don't expect to go off and build an entire e-commerce system with only this class 
 under your belt.\n\nIt's important to note that there's a long gap from site design to actual implementation.  
 This course teaches you how to implement a design.  The user interface and overall user experience is a different 
 subject that isn't covered here...it's possible to do a great implementation of an absolutely abysmal design.  
 I speak from experience.  :)\n\nAs I said above, if you're a novice, a relative newcomer or just an experienced web 
 designer who wants a refresher course, this is a good way to do it."
"""
sample_input = json.dumps({
    'data': [comments]
})
response = requests.post(uri, data=sample_input, headers=headers)
print(response.json())

[4.0]


### Use Service Run Function

In [87]:
print(service.run(sample_input))

[4.0]
